In [ ]:
# In total takes about 30 minutes for one experiment
from pathlib import Path
from brain_observatory_qc.pipeline_dev import paired_plane_registration as ppr
import h5py

from brain_observatory_qc.pipeline_dev import decrosstalk_roi_images_ica as dc_roi_images

decrosstalk_dir = Path(r'\\allen\programs\mindscope\workgroups\learning\pipeline_validation\decrosstalk')
data_dir = decrosstalk_dir / 'paired_registration_data'

In [ ]:
oeid = 1167237079
paired_id = ppr.get_paired_plane_id(oeid)
paired_reg_fn = data_dir / f'{paired_id}_paired_registered.h5'
decrosstalk_data, alpha_list, beta_list, mean_norm_mi_list = dc_roi_images.decrosstalk_movie_roi_image(oeid, paired_reg_fn)


In [ ]:
dc_fov_dir = data_dir / 'decrosstalk_roi_images'
if not dc_fov_dir.exists():
    dc_fov_dir.mkdir()
decrosstalk_fn = dc_fov_dir / f'{oeid}_decrosstalk.h5'
with h5py.File(decrosstalk_fn, 'w') as f:
    f.create_dataset('data', data=decrosstalk_data)
    f.create_dataset('alpha_list', data=alpha_list)
    f.create_dataset('beta_list', data=beta_list)